### <span style="color:blue"> *---------------------------------------*</span>
# <span style="color:blue"> *UDV project - SVD based pruning*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
# Imports libraries

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import numpy as np

from torchvision import transforms
from torchvision import models
from collections import OrderedDict

import pickle
import time
import matplotlib.pyplot as plt

In [ ]:
# Set the global seed for reproducibility

public_seed = 529
torch.manual_seed(public_seed)
print("Current Seed is {0}".format(public_seed))

In [ ]:
# Obtain device

from udvFunctions.udvDevice import get_device
device = get_device()

In [ ]:
# Load dataset

from udvFunctions.udvClassDataset import MNIST_Pre

In [ ]:
# Prepare the single-connection layer that carry weight matrix 'w'

from udvFunctions.udvDiagonalLayer import D_singleConnection   

In [ ]:
# Set the validation loop

from udvFunctions.udvClassVal import class_valLoop

In [ ]:
# Other customised functions

from udvFunctions.udvOtherFunctions import store_metrics, take_avg, check_shapes

### <span style="color:blue"> *---------------------------------------*</span>
# <span style="color:blue"> *Set SVD-based pruning test (Without re-train):*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
# Dataset selection
from udvFunctions.udvOtherFunctions import obtain_current_directory

dataset_choice = 0 # 0 points MNIST dataset
torch.manual_seed(public_seed)
if dataset_choice == 0:
    folder_list = obtain_current_directory(endString = "PreTrue") 
    data_path = "./MNIST"
    train_dataset, val_dataset, num_output = MNIST_Pre(load_All = True, data_path = data_path)  
    pre_path = "./"
    print("load the MNIST dataset")

else:
    print("Wrong selection on dataset")

model_list = ["/SingleLayer/Seed_1_finalEpoch_70_model_0.pt",
              "/SingleLayer/Seed_1_finalEpoch_70_model_1.pt",
              "/SingleLayer/Seed_1_finalEpoch_70_model_2.pt",
              "/SingleLayer/Seed_1_finalEpoch_70_model_3.pt"]


In [ ]:
# Warning: raise Error if folder_list is not match
# Revise the folder_list according to the baseline results
from udvFunctions.udvOtherFunctions import CPU_Unpickler, non_Value, pruning_list, same_saving
from udvFunctions.udvClassObtainModels import revised_model, pruning_model

for drop_file_index in range(len(folder_list)): 
    # Set path and open file
    store_file_path = pre_path + folder_list[drop_file_index] + '/SingleLayer/results.pkl'
    print("current pickle file is: ", store_file_path)

    with open(store_file_path, 'rb') as file:
        #variables = CPU_Unpickler(file).load()
        variables = pickle.load(file)
        
    # Read parameters
    num_epochs = variables['num_epochs']
    num_seeds = variables['num_seeds']
    batch_size = variables['batch_size']

    trans_model = variables['trans_model']
    optimiser_name = variables['optimiser_name']
    learning_rate = variables['learning_rate']

    num_input = variables['num_input']
    num_hidden_1 = variables['num_hidden_1']
    num_output = variables['num_output']   

    u_1_V_model_0 = variables['u_1_V_model_0']
    w_1_V_model_0 = variables['w_1_V_model_0']
    v_1_V_model_0 = variables['v_1_V_model_0']

    u_1_V_model_1 = variables['u_1_V_model_1']
    w_1_V_model_1 = variables['w_1_V_model_1']
    v_1_V_model_1 = variables['v_1_V_model_1']

    u_1_M_model_2 = variables['u_1_M_model_2']
    w_1_M_model_2 = variables['w_1_M_model_2']
    u_2_M_model_2 = variables['u_2_M_model_2']

    u_1_M_model_3 = variables['u_1_M_model_3']
    w_1_M_model_3 = variables['w_1_M_model_3']
    u_2_M_model_3 = variables['u_2_M_model_3']
    
    seed_index_list = list(range(num_seeds))

    num_workers = 4
    train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = num_workers)
    val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = num_workers)
    
    loss_fn = nn.CrossEntropyLoss()

    # Re-producible setting
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Create pruning list and blank space for saving validation loss
    # This code gradually decrease number of hidden neurons and obtain validation loss
    neurons_list = pruning_list(start_number = num_hidden_1, deduce_factor = 0.9) 
    val_losses_m_0 = [0] * (len(neurons_list) + 1)
    val_losses_m_1 = [0] * (len(neurons_list) + 1)
    val_losses_m_2 = [0] * (len(neurons_list) + 1)
    val_losses_m_3 = [0] * (len(neurons_list) + 1)
    val_acces_m_0 = [0] * (len(neurons_list) + 1)
    val_acces_m_1 = [0] * (len(neurons_list) + 1)
    val_acces_m_2 = [0] * (len(neurons_list) + 1)
    val_acces_m_3 = [0] * (len(neurons_list) + 1)

    # The SVD-based pruning results will be averaged by the same number of seeds as baseline code
    for seed_index in seed_index_list:
    
    # =======================Model_0=================Model_0====================================
        # If "NaN" is involved, no need to do the validation
        if not (non_Value(u_1_V_model_0) or non_Value(w_1_V_model_0) or non_Value(v_1_V_model_0)):    
            
            # Set the pt file path and load model
            store_pt_path = pre_path + folder_list[drop_file_index] + model_list[0]
            model = revised_model(name = trans_model,
                                  train_features = True,
                                  pre_trained = True, 
                                  model_order = 0, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
            # Model for validation (Not trainable)
            model.load_state_dict(torch.load(store_pt_path))
            model.to(device)
            
            # Compare (Verify) the saved weights from pickle and pt files
            same_saving(u1 = u_1_V_model_0[seed_index], w1 = w_1_V_model_0[seed_index], v1 = v_1_V_model_0[seed_index],
                        u2 = model.classifier.fc1.weight, w2 = model.classifier.diag1.weight, v2 = model.classifier.fc2.weight)
            
            # Obtain the baseline (Original number of hidden neurons)
            val_loss_drop, val_acc_drop = class_valLoop(model = model,
                                                        loss_fn = loss_fn,
                                                        val_loader = val_dataloader,
                                                        device = device)
            val_losses_m_0[0] += val_loss_drop
            val_acces_m_0[0] += val_acc_drop
            del model
            
            # SVD of model_0: Vector_uwv - UDV-v1
            u1w1_m_0 = (torch.mul(w_1_V_model_0[seed_index].t(), u_1_V_model_0[seed_index])).t()
            U_m_0, S_m_0, Vh_m_0 = torch.linalg.svd(u1w1_m_0, full_matrices = False)
            
            # Gradually reduce number of hidden neurons
            for num_hidden_new_index, num_hidden_new in enumerate(neurons_list):
                U_m_0_drop = U_m_0.clone().detach()[:,:num_hidden_new]
                S_m_0_drop = S_m_0.clone().detach()[:num_hidden_new]
                Vh_m_0_drop = Vh_m_0.clone().detach()[:num_hidden_new,:]
                
                #Re-load the model
                model = revised_model(name = trans_model,
                                      train_features = True,
                                      pre_trained = True, 
                                      model_order = 0, 
                                      num_input = num_input,
                                      num_hidden_1 = num_hidden_1, 
                                      num_output = num_output)
                
                # Model for validation (Not trainable) and set new top layers
                model = pruning_model(model = model, 
                                      name = trans_model, 
                                      num_input = num_input,
                                      num_hidden_new = num_hidden_new,
                                      num_output = num_output,
                                      ptPath = store_pt_path,
                                      device = device)
                
                # Verify the weight matrices' shape
                with torch.no_grad():
                    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = U_m_0_drop.t())
                    check_shapes(model_weight = model.classifier.diag1.weight, list_sample = S_m_0_drop.unsqueeze(0))
                    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = (v_1_V_model_0[seed_index].clone().detach())@(Vh_m_0_drop.t()))
                    model.classifier.fc1.weight = nn.Parameter(U_m_0_drop.t())
                    model.classifier.diag1.weight = nn.Parameter(S_m_0_drop.unsqueeze(0))
                    model.classifier.fc2.weight = nn.Parameter((v_1_V_model_0[seed_index].clone().detach())@(Vh_m_0_drop.t()))
                
                # Obtain the validation metrics 
                val_loss_drop, val_acc_drop = class_valLoop(model = model,
                                                            loss_fn = loss_fn,
                                                            val_loader = val_dataloader,
                                                            device = device)
                val_losses_m_0[num_hidden_new_index + 1] += val_loss_drop
                val_acces_m_0[num_hidden_new_index + 1] += val_acc_drop
                print("#neurons = {0} has been updated".format(num_hidden_new))
                
                del model, U_m_0_drop, S_m_0_drop, Vh_m_0_drop, val_loss_drop, val_acc_drop
            del u1w1_m_0, U_m_0, S_m_0, Vh_m_0
        
        else:
            # If NaN things involved, we still maintrain the saving structuer
            val_losses_m_0[0] += 10000
            val_acces_m_0[0] += 10000
            for num_hidden_new_index, num_hidden_new in enumerate(neurons_list):
                val_losses_m_0[num_hidden_new_index + 1] += 10000
                val_acces_m_0[num_hidden_new_index + 1] += 10000
        print("finish test model 0\n\n\n")
    # =======================Model_0======DONE=======Model_0====================================
    
    # =======================Model_1=================Model_1====================================
        # UDV-v2
        if not (non_Value(u_1_V_model_1) or non_Value(w_1_V_model_1) or non_Value(v_1_V_model_1)):    
            store_pt_path = pre_path + folder_list[drop_file_index] + model_list[1]
            model = revised_model(name = trans_model,
                                  train_features = True,
                                  pre_trained = True, 
                                  model_order = 1, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)

            model.load_state_dict(torch.load(store_pt_path))
            model.to(device)
            
            same_saving(u1 = u_1_V_model_1[seed_index], w1 = w_1_V_model_1[seed_index], v1 = v_1_V_model_1[seed_index],
                        u2 = model.classifier.fc1.weight, w2 = model.classifier.diag1.weight, v2 = model.classifier.fc2.weight)
            
            val_loss_drop, val_acc_drop = class_valLoop(model = model,
                                                        loss_fn = loss_fn,
                                                        val_loader = val_dataloader,
                                                        device = device)
            val_losses_m_1[0] += val_loss_drop
            val_acces_m_1[0] += val_acc_drop
            del model
            
            u1w1_m_1 = (torch.mul(w_1_V_model_1[seed_index].t(), u_1_V_model_1[seed_index])).t()
            U_m_1, S_m_1, Vh_m_1 = torch.linalg.svd(u1w1_m_1, full_matrices = False)

            for num_hidden_new_index, num_hidden_new in enumerate(neurons_list):
                U_m_1_drop = U_m_1.clone().detach()[:,:num_hidden_new]
                S_m_1_drop = S_m_1.clone().detach()[:num_hidden_new]
                Vh_m_1_drop = Vh_m_1.clone().detach()[:num_hidden_new,:]
                
                model = revised_model(name = trans_model,
                                      train_features = True,
                                      pre_trained = True, 
                                      model_order = 1, 
                                      num_input = num_input,
                                      num_hidden_1 = num_hidden_1, 
                                      num_output = num_output)
                model = pruning_model(model = model, 
                                      name = trans_model, 
                                      num_input = num_input,
                                      num_hidden_new = num_hidden_new,
                                      num_output = num_output,
                                      ptPath = store_pt_path,
                                      device = device)
                
                with torch.no_grad():
                    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = U_m_1_drop.t())
                    check_shapes(model_weight = model.classifier.diag1.weight, list_sample = S_m_1_drop.unsqueeze(0))
                    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = (v_1_V_model_1[seed_index].clone().detach())@(Vh_m_1_drop.t()))
                    model.classifier.fc1.weight = nn.Parameter(U_m_1_drop.t())
                    model.classifier.diag1.weight = nn.Parameter(S_m_1_drop.unsqueeze(0))
                    model.classifier.fc2.weight = nn.Parameter((v_1_V_model_1[seed_index].clone().detach())@(Vh_m_1_drop.t()))
                
                val_loss_drop, val_acc_drop = class_valLoop(model = model,
                                                            loss_fn = loss_fn,
                                                            val_loader = val_dataloader,
                                                            device = device)
                val_losses_m_1[num_hidden_new_index + 1] += val_loss_drop
                val_acces_m_1[num_hidden_new_index + 1] += val_acc_drop
                print("#neurons = {0} has been updated".format(num_hidden_new))
                
                del model, U_m_1_drop, S_m_1_drop, Vh_m_1_drop, val_loss_drop, val_acc_drop
            del u1w1_m_1, U_m_1, S_m_1, Vh_m_1
        
        else:
            val_losses_m_1[0] += 10000
            val_acces_m_1[0] += 10000
            for num_hidden_new_index, num_hidden_new in enumerate(neurons_list):
                val_losses_m_1[num_hidden_new_index + 1] += 10000
                val_acces_m_1[num_hidden_new_index + 1] += 10000
        print("finish test model 1\n\n\n")
    # =======================Model_1======DONE=======Model_1==================================== 

    # =======================Model_2==================Model_2====================================
        # UDV
        if not (non_Value(u_1_M_model_2) or non_Value(w_1_M_model_2) or non_Value(u_2_M_model_2)):    
            store_pt_path = pre_path + folder_list[drop_file_index] + model_list[2]
            model = revised_model(name = trans_model,
                                  train_features = True,
                                  pre_trained = True, 
                                  model_order = 2, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
            
            model.load_state_dict(torch.load(store_pt_path))
            model.to(device)
            
            same_saving(u1 = u_1_M_model_2[seed_index], w1 = w_1_M_model_2[seed_index], v1 = u_2_M_model_2[seed_index],
                        u2 = model.classifier.fc1.weight, w2 = model.classifier.diag1.weight, v2 = model.classifier.fc2.weight)
            
            val_loss_drop, val_acc_drop = class_valLoop(model = model,
                                                        loss_fn = loss_fn,
                                                        val_loader = val_dataloader,
                                                        device = device)
            val_losses_m_2[0] += val_loss_drop
            val_acces_m_2[0] += val_acc_drop
            del model
            
            u1w1_m_2 = (torch.mul(w_1_M_model_2[seed_index].t(), u_1_M_model_2[seed_index])).t()
            U_m_2, S_m_2, Vh_m_2 = torch.linalg.svd(u1w1_m_2, full_matrices = False)
            
            for num_hidden_new_index, num_hidden_new in enumerate(neurons_list):
                U_m_2_drop = U_m_2.clone().detach()[:,:num_hidden_new]
                S_m_2_drop = S_m_2.clone().detach()[:num_hidden_new]
                Vh_m_2_drop = Vh_m_2.clone().detach()[:num_hidden_new,:]
                
                model = revised_model(name = trans_model,
                                      train_features = True,
                                      pre_trained = True, 
                                      model_order = 2, 
                                      num_input = num_input,
                                      num_hidden_1 = num_hidden_1, 
                                      num_output = num_output)
                
                model = pruning_model(model = model, 
                                      name = trans_model, 
                                      num_input = num_input,
                                      num_hidden_new = num_hidden_new,
                                      num_output = num_output,
                                      ptPath = store_pt_path,
                                      device = device)
                
                with torch.no_grad():
                    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = U_m_2_drop.t())
                    check_shapes(model_weight = model.classifier.diag1.weight, list_sample = S_m_2_drop.unsqueeze(0))
                    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = (u_2_M_model_2[seed_index].clone().detach())@(Vh_m_2_drop.t()))
                    model.classifier.fc1.weight = nn.Parameter(U_m_2_drop.t())
                    model.classifier.diag1.weight = nn.Parameter(S_m_2_drop.unsqueeze(0))
                    model.classifier.fc2.weight = nn.Parameter((u_2_M_model_2[seed_index].clone().detach())@(Vh_m_2_drop.t()))
                
                val_loss_drop, val_acc_drop = class_valLoop(model = model,
                                                            loss_fn = loss_fn,
                                                            val_loader = val_dataloader,
                                                            device = device)
                val_losses_m_2[num_hidden_new_index + 1] += val_loss_drop
                val_acces_m_2[num_hidden_new_index + 1] += val_acc_drop
                print("#neurons = {0} has been updated".format(num_hidden_new))
                
                del model, U_m_2_drop, S_m_2_drop, Vh_m_2_drop, val_loss_drop, val_acc_drop
            del u1w1_m_2, U_m_2, S_m_2, Vh_m_2
        
        else:
            val_losses_m_2[0] += 10000
            val_acces_m_2[0] += 10000
            for num_hidden_new_index, num_hidden_new in enumerate(neurons_list):
                val_losses_m_2[num_hidden_new_index + 1] += 10000
                val_acces_m_2[num_hidden_new_index + 1] += 10000
        print("finish test model 2\n\n\n")
    # =======================Model_2======DONE=======Model_2==================================== 

    # =======================Model_3==================Model_3====================================
        # UDV-s
        if not (non_Value(u_1_M_model_3) or non_Value(w_1_M_model_3) or non_Value(u_2_M_model_3)):    
            store_pt_path = pre_path + folder_list[drop_file_index] + model_list[3]
            model = revised_model(name = trans_model,
                                  train_features = True,
                                  pre_trained = True, 
                                  model_order = 3, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
            
            model.load_state_dict(torch.load(store_pt_path))
            model.to(device)
            
            same_saving(u1 = u_1_M_model_3[seed_index], w1 = w_1_M_model_3[seed_index], v1 = u_2_M_model_3[seed_index],
                        u2 = model.classifier.fc1.weight, w2 = model.classifier.diag1.weight, v2 = model.classifier.fc2.weight)
            
            val_loss_drop, val_acc_drop = class_valLoop(model = model,
                                                        loss_fn = loss_fn,
                                                        val_loader = val_dataloader,
                                                        device = device)
            val_losses_m_3[0] += val_loss_drop
            val_acces_m_3[0] += val_acc_drop
            del model
            
            u1w1_m_3 = (torch.mul(w_1_M_model_3[seed_index].t(), u_1_M_model_3[seed_index])).t()
            U_m_3, S_m_3, Vh_m_3 = torch.linalg.svd(u1w1_m_3, full_matrices = False)
            
            for num_hidden_new_index, num_hidden_new in enumerate(neurons_list):
                U_m_3_drop = U_m_3.clone().detach()[:,:num_hidden_new]
                S_m_3_drop = S_m_3.clone().detach()[:num_hidden_new]
                Vh_m_3_drop = Vh_m_3.clone().detach()[:num_hidden_new,:]
                
                model = revised_model(name = trans_model,
                                      train_features = True,
                                      pre_trained = True, 
                                      model_order = 3, 
                                      num_input = num_input,
                                      num_hidden_1 = num_hidden_1, 
                                      num_output = num_output)
                
                model = pruning_model(model = model, 
                                      name = trans_model, 
                                      num_input = num_input,
                                      num_hidden_new = num_hidden_new,
                                      num_output = num_output,
                                      ptPath = store_pt_path,
                                      device = device)
                
                with torch.no_grad():
                    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = U_m_3_drop.t())
                    check_shapes(model_weight = model.classifier.diag1.weight, list_sample = S_m_3_drop.unsqueeze(0))
                    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = (u_2_M_model_3[seed_index].clone().detach())@(Vh_m_3_drop.t()))
                    model.classifier.fc1.weight = nn.Parameter(U_m_3_drop.t())
                    model.classifier.diag1.weight = nn.Parameter(S_m_3_drop.unsqueeze(0))
                    model.classifier.fc2.weight = nn.Parameter((u_2_M_model_3[seed_index].clone().detach())@(Vh_m_3_drop.t()))
                
                val_loss_drop, val_acc_drop = class_valLoop(model = model,
                                                            loss_fn = loss_fn,
                                                            val_loader = val_dataloader,
                                                            device = device)
                val_losses_m_3[num_hidden_new_index + 1] += val_loss_drop
                val_acces_m_3[num_hidden_new_index + 1] += val_acc_drop
                print("#neurons = {0} has been updated".format(num_hidden_new))
                
                del model, U_m_3_drop, S_m_3_drop, Vh_m_3_drop, val_loss_drop, val_acc_drop
            del u1w1_m_3, U_m_3, S_m_3, Vh_m_3
        
        else:
            val_losses_m_3[0] += 10000
            val_acces_m_3[0] += 10000
            for num_hidden_new_index, num_hidden_new in enumerate(neurons_list):
                val_losses_m_3[num_hidden_new_index + 1] += 10000
                val_acces_m_3[num_hidden_new_index + 1] += 10000
        print("finish test model 3\n\n\n")
    # =======================Model_3======DONE=======Model_3==================================== 
    
    # Avergae the accumulate validation loss/acc
    for avg_index in range(len(val_losses_m_0)):
        val_losses_m_0[avg_index] /= num_seeds
        val_losses_m_1[avg_index] /= num_seeds
        val_losses_m_2[avg_index] /= num_seeds
        val_losses_m_3[avg_index] /= num_seeds
        val_acces_m_0[avg_index] /= num_seeds
        val_acces_m_1[avg_index] /= num_seeds
        val_acces_m_2[avg_index] /= num_seeds
        val_acces_m_3[avg_index] /= num_seeds

    # Convert data to percentage compared to the baseline
    val_losses_m_0_plot = [(x - val_losses_m_0[0]) / val_losses_m_0[0] for x in val_losses_m_0]
    val_losses_m_1_plot = [(x - val_losses_m_1[0]) / val_losses_m_1[0] for x in val_losses_m_1]
    val_losses_m_2_plot = [(x - val_losses_m_2[0]) / val_losses_m_2[0] for x in val_losses_m_2]
    val_losses_m_3_plot = [(x - val_losses_m_3[0]) / val_losses_m_3[0] for x in val_losses_m_3]
    # val_acces_m_0_plot = [(x - val_acces_m_0[0]) / val_acces_m_0[0] for x in val_acces_m_0]
    # val_acces_m_1_plot = [(x - val_acces_m_1[0]) / val_acces_m_1[0] for x in val_acces_m_1]
    # val_acces_m_2_plot = [(x - val_acces_m_2[0]) / val_acces_m_2[0] for x in val_acces_m_2]
    # val_acces_m_3_plot = [(x - val_acces_m_3[0]) / val_acces_m_3[0] for x in val_acces_m_3]

    # =======================WriteToFile=================WriteToFile============================     

    with open('XXXXXX.txt', 'a+') as file: # Create or add text to the file
        file.seek(0, 2) # Find the end of the file
        for write_index in range(0, len(neurons_list) + 1):
            file.write("{0}\t".format(val_losses_m_0[write_index]))
            file.write("{0}\t".format(val_losses_m_1[write_index]))
            file.write("{0}\t".format(val_losses_m_2[write_index]))
            file.write("{0}\t".format(val_losses_m_3[write_index]))
            if write_index == 0:
                file.write("{0}\t".format(num_hidden_1))
            else:
                file.write("{0}\t".format(neurons_list[write_index - 1]))
            file.write("{0}\t".format(val_losses_m_0_plot[write_index]))
            file.write("{0}\t".format(val_losses_m_1_plot[write_index]))
            file.write("{0}\t".format(val_losses_m_2_plot[write_index]))
            file.write("{0}\t".format(val_losses_m_3_plot[write_index]))
            if write_index == 0:
                file.write("{0}\t".format(num_hidden_1))
            else:
                file.write("{0}\t".format(neurons_list[write_index - 1]))
            file.write("{0}\t".format(val_acces_m_0[write_index]))
            file.write("{0}\t".format(val_acces_m_1[write_index]))
            file.write("{0}\t".format(val_acces_m_2[write_index]))
            file.write("{0}\t".format(val_acces_m_3[write_index]))
            file.write("\n")
        file.write("\n\n\n\n\n")
    print("The file has been run: ", store_file_path)
print("All done")